In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from uncertainties import unumpy as unp
import uncertainties
import seaborn as sb
import matplotlib.cm as cm
import matplotlib.ticker as ticker
import scipy
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from scipy.stats.mstats import gmean
import re
import geopandas as gpd
from itertools import combinations
from scipy.stats import ttest_ind
import cartopy.crs as ccrs
#import rioxarray
#import xarray as xr
#import rasterstats as rs
#import rasterio as rio
#from rasterio.warp import calculate_default_transform, reproject, Resampling

/Users/Yuvalr/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


## Old analysis

1. Read the data

In [2]:
raw_data = pd.read_excel('data/RawData.xlsx')
#raw_data = raw_data[~raw_data.units.str('trap')]

/Users/Yuvalr/opt/anaconda3/lib/python3.7/site-packages/openpyxl/worksheet/_reader.py:296: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


2. Focus on the sites which have competing data: both population (number of individuals) and biomass

In [3]:
raw_data.loc[:,'norm value'] = raw_data.loc[:,'numerical value']
raw_data.loc[raw_data.units=='mg/m^2 (wet weight)','norm value'] = raw_data.loc[raw_data.units=='mg/m^2 (wet weight)','norm value']*0.3 #convert the wet mass to an effective dry mass, by multiplying by 0.3

def unit_type(x):
    X=x.partition('/')[0]#[str(xi).partition('/')[0] for xi in x]
    return X

raw_data=raw_data[~raw_data.units.isna()] #temporary, to eliminate bad lines
raw_data.loc[:,'units type'] = raw_data.units.apply(unit_type) #units type distinguishes population to biomass measurements
raw_data.groupby('units type').site.nunique()


units type
individuals    440
mg             392
Name: site, dtype: int64

3. Remove partial measurements

In [4]:
metadata = pd.read_csv('data/groups_per_reference.csv')
data_with_meta = raw_data.merge(metadata,left_on='reference', right_on='Paper',how='left')

In [5]:
valid_data = data_with_meta.copy()
valid_data = valid_data[(valid_data['Standard groups']!='Microarthropods') | ((valid_data['Standard groups']=='Microarthropods') & (valid_data['sub-class'].isin(['Acari','Collembola'])))]

4. Classify into groups using "aggregated taxon" and "aggregated environment"

In [6]:
valid_data.loc[valid_data['sub-class']=='Acari','aggregated taxon'] = 'Acari'
valid_data.loc[valid_data['sub-class']=='Collembola','aggregated taxon'] = 'Collembola'
valid_data.loc[valid_data['super-family']=='Isoptera','aggregated taxon'] = 'Isoptera'
valid_data.loc[valid_data['family']=='Formicidae','aggregated taxon'] = 'Formicidae'
valid_data.loc[valid_data['aggregated taxon'].isna(),'aggregated taxon'] = 'Other'

In [7]:
soil_data = valid_data[valid_data['aggregated environment'] =='soil/litter']
canopy_data = valid_data[valid_data['aggregated environment'] =='plants']
surface_data = valid_data[valid_data['aggregated environment'] =='above ground']

In [8]:
soil_data.groupby('aggregated biome').site.nunique()#.site.nunique()
#surface_data.site.nunique()


aggregated biome
Boreal Forests/Taiga                                            28
Crops                                                           35
Deserts and Xeric Shrublands                                    12
Mediterranean Forests, Woodlands and Scrub                      21
Pasture                                                         41
Shrubland/Grassland                                             13
Temperate Forests                                               97
Temperate Grasslands, Savannas and Shrublands                   38
Tropical and Subtropical Forests                                86
Tropical and Subtropical Grasslands, Savannas and Shrublands    27
Tundra                                                          65
Name: site, dtype: int64

5. Remove measurements with unknown biomes (ants)

In [9]:
soil_data = soil_data[soil_data['aggregated biome'] != 'Shrubland/Grassland']#Drop the ants measurements with unknown biomes

7. Calculate means

In [10]:
#Average the soil data over each taxon in each site, then sum all taxons in each site according to the aggregated groups and data type they are in.
soil_site_taxa_mean = soil_data.groupby(['units type','aggregated taxon','aggregated biome','site','taxon'])['norm value'].mean().reset_index()
soil_site_data = soil_site_taxa_mean.groupby(['units type','aggregated taxon','aggregated biome','site'])['norm value'].sum().reset_index()
#soil_site_data = soil_site_taxa_mean.groupby(['units type','aggregated taxon','aggregated biome','site'])['norm value'].sum().reset_index()

#Divide into the two types of measurements
soil_site_data_pop = soil_site_data[soil_site_data['units type']=='individuals']
soil_site_data_pop.rename(columns={'norm value': 'population density'}, inplace=True)

soil_site_data_mass = soil_site_data[soil_site_data['units type']=='mg']
soil_site_data_mass.rename(columns={'norm value': 'mass density'}, inplace=True)

#Construct a new dataframe, where we keep only the sites with both measurement types
soil_site_data_comb = pd.merge( soil_site_data_pop, soil_site_data_mass, on=["aggregated taxon","aggregated biome","site"], how="inner", validate="one_to_one" )

#calculate the mass of an individual per site, in units of mg/ind
soil_site_data_comb.loc[:,'ind mass']=soil_site_data_comb.loc[:,'mass density']/soil_site_data_comb.loc[:,'population density']

soil_site_data_comb.pivot_table(index='aggregated taxon',columns='aggregated biome',values='ind mass', aggfunc=['mean','count'])
#soil_site_data_comb = soil_site_data_comb.unstack().reset_index().pivot_table(index='aggregated taxon', columns=['aggregated biome','level_0'],values=0,aggfunc=sum)
#soil_site_data_comb.columns = soil_site_data_comb.columns.set_levels(['Mean','N'],1)

/Users/Yuvalr/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:5042: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


mean                                        \
aggregated biome Boreal Forests/Taiga     Crops   Pasture Temperate Forests   
aggregated taxon                                                              
Acari                        0.003206       NaN       NaN          0.015937   
Collembola                   0.004459       NaN       NaN          0.015480   
Formicidae                        NaN  0.343756  0.644897          2.278229   
Isoptera                          NaN  0.819060  0.672940          0.680539   
Other                        0.345232  2.514433  3.820162          3.269194   

                                                                \
aggregated biome Temperate Grasslands, Savannas and Shrublands   
aggregated taxon                                                 
Acari                                                 0.001764   
Collembola                                            0.002450   
Formicidae                                            0.548186   
Isoptera                                              0.289569   
Other                                                 3.153510   

                                                   \
aggregated biome Tropical and Subtropical Forests   
aggregated taxon                                    
Acari                                    0.001765   
Collembola                               0.008673   
Formicidae                               0.424516   
Isoptera                                 1.069180   
Other                                    4.044042   

                                                                               \
aggregated biome Tropical and Subtropical Grasslands, Savannas and Shrublands   
aggregated taxon                                                                
Acari                                                      0.002253             
Collembola                                                 0.004211             
Formicidae                                                 0.784287             
Isoptera                                                   1.665947             
Other                                                      3.133116             

                                          count                \
aggregated biome    Tundra Boreal Forests/Taiga Crops Pasture   
aggregated taxon                                                
Acari             0.005766                 17.0   NaN     NaN   
Collembola        0.006051                 17.0   NaN     NaN   
Formicidae             NaN                  NaN   6.0    10.0   
Isoptera               NaN                  0.0   9.0    16.0   
Other             1.242991                 15.0   7.0    11.0   

                                    \
aggregated biome Temperate Forests   
aggregated taxon                     
Acari                         13.0   
Collembola                    13.0   
Formicidae                     5.0   
Isoptera                       3.0   
Other                         14.0   

                                                                \
aggregated biome Temperate Grasslands, Savannas and Shrublands   
aggregated taxon                                                 
Acari                                                      3.0   
Collembola                                                 7.0   
Formicidae                                                 4.0   
Isoptera                                                   1.0   
Other                                                     11.0   

                                                   \
aggregated biome Tropical and Subtropical Forests   
aggregated taxon                                    
Acari                                         4.0   
Collembola                                    4.0   
Formicidae                                   10.0   
Isoptera                                     32.0   
Other                                        16.0   

                 

In [11]:
def filter_outliers(x):
    if len(x)>3:
        STD=x.std()
        Mean=np.mean(x)
        x = x[x<(Mean+2*STD)]
        x = x[x>(Mean-2*STD)]
    return x

soil_ratio_filt = soil_site_data_comb.groupby(['aggregated taxon','aggregated biome'])['ind mass'].apply(filter_outliers)
soil_ratio_filt_full = soil_ratio_filt.reset_index().pivot_table(index='aggregated taxon',columns='aggregated biome',values='ind mass', aggfunc='mean')
soil_ratio_filt_total = soil_ratio_filt.reset_index().groupby('aggregated taxon')['ind mass'].mean()
soil_ratio_filt_full = pd.merge(soil_ratio_filt_full, soil_ratio_filt_total,on="aggregated taxon", how="inner", validate="one_to_one" )

soil_ratio_filt_full.rename(columns={'ind mass': 'Total average (mg/ind)'}, inplace=True)

#fill in the nans with the global averages
for clm in soil_ratio_filt_full.columns:
    soil_ratio_filt_full.loc[soil_ratio_filt_full[clm].isnull(),clm] = soil_ratio_filt_full['Total average (mg/ind)']

biomes = soil_site_data_pop.loc[:,'aggregated biome'].unique()

for biom in biomes: 
    if biom not in soil_ratio_filt_full.columns:
        soil_ratio_filt_full.loc[:,biom] = soil_ratio_filt_full.loc[:,'Total average (mg/ind)']
        
# soil_ratio_filt_full.to_csv('results/average_ind_mass_full.csv')##       

def print_ratio(x):
    return '{:.2e}'.format(x)

soil_ratio_filt_print = soil_ratio_filt_full.applymap(print_ratio)
#soil_ratio_filt_print.to_csv('results/table_average_ind_mass.csv')##

#soil_ratio_filt_pd = soil_ratio_filt.reset_index().groupby(["aggregated taxon","aggregated biome"]).mean()['ind mass'].reset_index()
#soil_ratio_filt_pd = soil_ratio_filt.reset_index().groupby(["aggregated taxon","aggregated biome"]).mean().drop(columns='level_2')
#soil_ratio_filt_pd.to_csv('results/pd_average_ind_mass.csv')##


#soil_ratio_filt_print
soil_ratio_filt_full.applymap(print_ratio)



,Boreal Forests/Taiga,Crops,Pasture,Temperate Forests,"Temperate Grasslands, Savannas and Shrublands",Tropical and Subtropical Forests,"Tropical and Subtropical Grasslands, Savannas and Shrublands",Tundra,Total average (mg/ind),Deserts and Xeric Shrublands,"Mediterranean Forests, Woodlands and Scrub"
aggregated taxon,,,,,,,,,,,
Acari,2.90e-03,4.72e-03,4.72e-03,8.40e-03,1.76e-03,1.77e-03,2.25e-03,4.95e-03,4.72e-03,4.72e-03,4.72e-03
Collembola,4.17e-03,5.58e-03,5.58e-03,9.67e-03,2.45e-03,8.67e-03,4.21e-03,5.12e-03,5.58e-03,5.58e-03,5.58e-03
Formicidae,6.98e-01,1.28e-01,6.45e-01,2.28e+00,5.48e-01,2.34e-01,7.84e-01,6.98e-01,6.98e-01,6.98e-01,6.98e-01
Isoptera,7.63e-01,8.19e-01,6.17e-01,6.81e-01,2.90e-01,7.95e-01,9.14e-01,7.63e-01,7.63e-01,7.63e-01,7.63e-01
Other,3.02e-01,2.51e+00,3.82e+00,1.52e+00,2.35e+00,2.62e+00,3.13e+00,2.87e-01,1.88e+00,1.88e+00,1.88e+00


In [12]:
#find the measurements where only population data is available
cond1 = soil_site_data_pop['site'].isin(soil_site_data_comb['site'])
cond2 = soil_site_data_pop['population density'].isin(soil_site_data_comb['population density'])
soil_site_data_pop_pure = soil_site_data_pop.drop(soil_site_data_pop[cond1 & cond2].index )#measurements with only population measurements, and no mass measurements, based on site name and numerical value

In [13]:
#convert population to effective mass
soil_site_data_pop_pure.loc[:,'eff. mass_B'] = np.nan # eff. mass_B will use taxon and biome level average mass of individual
soil_site_data_pop_pure.loc[:,'eff. mass_G'] = np.nan # eff. mass_G will use global average for each taxon
for ii in soil_site_data_pop_pure.index:
    if soil_site_data_pop_pure.loc[ii,'aggregated taxon'] != 'Other':
        soil_site_data_pop_pure.loc[ii,'eff. mass_B'] = soil_site_data_pop_pure.loc[ii,'population density']* soil_ratio_filt_full.loc[soil_site_data_pop_pure.loc[ii,'aggregated taxon'],soil_site_data_pop_pure.loc[ii,'aggregated biome']]
        soil_site_data_pop_pure.loc[ii,'eff. mass_G'] = soil_site_data_pop_pure.loc[ii,'population density']* soil_ratio_filt_full.loc[soil_site_data_pop_pure.loc[ii,'aggregated taxon'],'Total average (mg/ind)']

#contains all the measurements, converted into mass
soil_site_data_mass_all = pd.merge(soil_site_data_mass,soil_site_data_pop_pure,on=["aggregated taxon","aggregated biome","site"], how="outer")
#mass_B/G columns are a combination of the measured mass with the effective mass where no direct measurement is given (_B and _G are for biome level or global level)
indx_mass = ~np.isnan(soil_site_data_mass_all.loc[:,'mass density']) #index of the mass measurements

soil_site_data_mass_all.loc[:,'mass_B'] = np.nan
soil_site_data_mass_all.loc[:,'mass_G'] = np.nan
soil_site_data_mass_all.loc[indx_mass,'mass_B'] = soil_site_data_mass_all.loc[indx_mass,'mass density']
soil_site_data_mass_all.loc[~indx_mass,'mass_B'] = soil_site_data_mass_all.loc[~indx_mass,'eff. mass_B']
soil_site_data_mass_all.loc[indx_mass,'mass_G'] = soil_site_data_mass_all.loc[indx_mass,'mass density']
soil_site_data_mass_all.loc[~indx_mass,'mass_G'] = soil_site_data_mass_all.loc[~indx_mass,'eff. mass_G']

soil_site_data_mass_all = soil_site_data_mass_all[~np.isnan(soil_site_data_mass_all.loc[:,'mass_G'])] # remove measurements of "other" taxa, and where there is no direct biomass measurement
#indx_pop_other = np.isnan(soil_site_data_mass_all.loc[:,'mass_G']) #index of measurements of "other" taxa, and where there is no direct biomass measurement

#soil_site_data_mass_all.groupby(["aggregated biome","aggregated taxon"]).count()

## Sensitiity analysis - look at the variability between biogeographical realms in the same biome

In [14]:
# load processed site data
soil_site_data_mass_all = pd.read_csv('results/processed_site_data_mean.csv',index_col=0)
soil_site_data_mass_all_std = pd.read_csv('results/processed_site_data_std.csv',index_col=0)
raw_data = pd.read_excel('data/RawData.xlsx')
raw_data.loc[:,'norm value'] = raw_data.loc[:,'numerical value']
raw_data.loc[raw_data.units=='mg/m^2 (wet weight)','norm value'] = raw_data.loc[raw_data.units=='mg/m^2 (wet weight)','norm value']*0.3 #convert the wet mass to an effective dry mass, by multiplying by 0.3

def unit_type(x):
    X=x.partition('/')[0]#[str(xi).partition('/')[0] for xi in x]
    return X

raw_data=raw_data[~raw_data.units.isna()] #temporary, to eliminate bad lines
raw_data.loc[:,'units type'] = raw_data.units.apply(unit_type) #units type distinguishes population to biomass measurements


/Users/Yuvalr/opt/anaconda3/lib/python3.7/site-packages/openpyxl/worksheet/_reader.py:296: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [15]:
def parse_latlon(s):
    ''' Parse a string s which contains either a longitude or a latitude and convert it into Degree Minute Second vector'''
    
    if len(s.split('°')) >1:
        deg = s.split('°')[0]
        rest = s.split('°')[1].strip()
        if len(rest.split("''"))>1:
            rest = rest.split("''")[0].split("'")
            minute = rest[0]
            sec = rest[1]
            return [deg,minute,sec]
        elif len(rest.split('"'))>1:
            rest = rest.split('"')[0].split("'")
            minute = rest[0]
            sec = rest[1]
            return [float(deg),float(minute),float(sec)]
        elif len(rest.split("'"))>1:
            minute = rest.split("'")[0]
            return [float(deg),float(minute),None]
        elif len(rest) == 0:
            return [float(deg),None,None]


def dms_to_dd(data):
    ''' Converts Degree Minute Second vector into Decimal Degrees coordinates'''
    
    return pd.Series(np.nansum([data['degrees'],np.sign(data['degrees'])*data['minutes']/60.,np.sign(data['degrees'])*data['seconds']/3600],axis=0),index=['lat','lon'])

def convert(s):
    ''' Convert a coordinate string s into a pandas Series of latitude and longitude in Decimal Degrees format'''
    try:
        pat = "^,*\s*"
        p = re.compile(pat)
        if len(s.split('N'))>1:
            latstr = s.split('N')[0]
            lonstr = re.sub(p,'',s.split('N')[1])
        elif len(s.split('S'))>1:
            latstr = s.split('S')[0]
            lonstr = re.sub(p,'',s.split('S')[1])
            latstr = '-'+latstr
        else:
             return None   
        if lonstr.find('W') != -1:
            lonstr = '-'+lonstr
        lonstr = lonstr.split('E')[0]
        lonstr = lonstr.split('W')[0]

        lat = np.array(parse_latlon(latstr))
        lon = np.array(parse_latlon(lonstr))
        res = dms_to_dd(pd.DataFrame([lat,lon],columns=['degrees','minutes','seconds']).astype(float))
    except Exception as e:
        res = 'error'
        return pd.Series([res,res],index=['lat','lon'])
    return res

In [16]:
soil_site_data_mass_all_coord = soil_site_data_mass_all.merge(raw_data[~raw_data.duplicated(['site','coordinates','synthetic coordinates','country'])][['site','synthetic coordinates','coordinates','country']],on='site')
soil_site_data_mass_all_coord['harmonized_coordinates'] = soil_site_data_mass_all_coord['coordinates']
soil_site_data_mass_all_coord.loc[~soil_site_data_mass_all_coord['synthetic coordinates'].isna(),'harmonized_coordinates'] = soil_site_data_mass_all_coord.loc[~soil_site_data_mass_all_coord['synthetic coordinates'].isna(),'synthetic coordinates'] #Merge

#site_locations = data.loc[:,['site','coordinates']].drop_duplicates()
site_locations = soil_site_data_mass_all_coord.loc[:,['site','harmonized_coordinates','aggregated taxon']].drop_duplicates()
site_with_coords = site_locations[(~site_locations.harmonized_coordinates.isna()) & (site_locations.harmonized_coordinates != 0)]
site_with_coords[['lat','lon']] = site_with_coords.loc[:,'harmonized_coordinates'].apply(convert)
site_with_coords = site_with_coords[site_with_coords.lat!='error']

/Users/Yuvalr/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3636: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [17]:
biogeo_realms = gpd.read_file('data/terr_eco_regions/tnc_terr_ecoregions.shp')

In [18]:
site_with_coords_gpd = gpd.GeoDataFrame(site_with_coords, geometry=gpd.points_from_xy(site_with_coords.lon,site_with_coords.lat))
site_with_coords_gpd.crs = biogeo_realms.crs
site_with_coords_gpd = gpd.tools.sjoin(site_with_coords_gpd,biogeo_realms)
site_with_coords_gpd = site_with_coords_gpd.drop('index_right',axis=1)
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

site_with_coords_gpd = gpd.tools.sjoin(site_with_coords_gpd,world.dissolve('continent').reset_index(),how='left')

In [19]:
soil_site_data_mass_all_realm = soil_site_data_mass_all.merge(site_with_coords_gpd[['site','WWF_REALM2']],on='site')
soil_site_data_mass_all_std_realm = soil_site_data_mass_all_std.merge(site_with_coords_gpd[['site','WWF_REALM2']],on='site')

In [20]:

site_with_coords_gpd.groupby('continent').site.nunique()


continent
Africa            60
Asia              48
Europe           101
North America    109
Oceania           12
South America     37
Name: site, dtype: int64

In [21]:
site_biome_realm_counts = soil_site_data_mass_all_realm.groupby(['aggregated taxon','aggregated biome','WWF_REALM2']).site.nunique()
taxa_biome_for_analysis = site_biome_realm_counts[site_biome_realm_counts>=10].groupby(['aggregated taxon','aggregated biome']).count()
taxa_biome_for_analysis = taxa_biome_for_analysis[taxa_biome_for_analysis>1]

### T-test analysis comparing the mean biomass density of the samples within the realm to other realms in the biome

In [22]:
sen_analysis_res = pd.DataFrame()

def ttest_run(c1, c2,df,index,measure='mass_B'):
    results = ttest_ind(np.log(df.loc[df.WWF_REALM2==c1,measure]), np.log(df.loc[df.WWF_REALM2==c2,measure]))
    res = pd.DataFrame({'categ1': c1,
                       'categ2': c2,
                       'tstat': results.statistic,
                       'pvalue': results.pvalue}, 
                       index = pd.MultiIndex.from_tuples([i]))    
    return res

for i, row in taxa_biome_for_analysis.iteritems():
    df = soil_site_data_mass_all_realm[(soil_site_data_mass_all_realm['aggregated taxon']==i[0]) & (soil_site_data_mass_all_realm['aggregated biome']==i[1])]
    df_list = [ttest_run(k, j,df,i) for k, j in combinations(df.WWF_REALM2.unique().tolist(), 2)]
    sen_analysis_res = sen_analysis_res.append(pd.concat(df_list))

In [23]:
sen_analysis_res

categ1      categ2     tstat  \
Acari      Temperate Forests                 Palearctic    Nearctic -2.886879   
           Tundra                              Nearctic  Palearctic -6.875559   
           Tundra                              Nearctic   Antarctic  0.923109   
           Tundra                            Palearctic   Antarctic  4.124744   
Collembola Temperate Forests                 Palearctic    Nearctic  0.232392   
Isoptera   Crops                             Indo-Malay   Neotropic -0.554866   
           Crops                             Indo-Malay  Afrotropic -1.383649   
           Crops                              Neotropic  Afrotropic -1.377901   
           Tropical and Subtropical Forests  Afrotropic   Neotropic  2.184353   
           Tropical and Subtropical Forests  Afrotropic  Indo-Malay -0.379681   
           Tropical and Subtropical Forests   Neotropic  Indo-Malay -3.047958   

                                                   pvalue  
Acari      Temperate Forests                 4.591577e-03  
           Tundra                            1.770679e-09  
           Tundra                            3.604739e-01  
           Tundra                            1.538747e-04  
Collembola Temperate Forests                 8.166198e-01  
Isoptera   Crops                             5.829680e-01  
           Crops                             1.867119e-01  
           Crops                             1.767391e-01  
           Tropical and Subtropical Forests  3.233720e-02  
           Tropical and Subtropical Forests  7.055005e-01  
           Tropical and Subtropical Forests  3.220941e-03

### Estimating the effect of splitting the biomes into realms on the final estimates of biomass

#### estimating the area of biomes and realms

In [24]:
# load the data on the biomes
ecoregion_gdf = gpd.read_file('data/terr_eco_regions/tnc_terr_ecoregions.shp')
aggregated_biome_data = pd.read_csv('data/aggregated_biomes_data_20210613.csv')
# group data based on aggregated biomes and realms and calculate the area for each
ecoregion_merged_gdf = ecoregion_gdf.merge(aggregated_biome_data[['biome','aggregated biome 1']],left_on='WWF_MHTNAM',right_on='biome')
aggregated_biomes_gpd = ecoregion_merged_gdf.dissolve(by=['aggregated biome 1','WWF_REALM2'])
crs = ccrs.Mollweide()
crs_proj4 = crs.proj4_init
aggregated_biomes_gpd_mol = aggregated_biomes_gpd.to_crs(crs_proj4)
biome_realm_area = aggregated_biomes_gpd_mol['geometry'].area
biome_realm_area.name = 'area'

/Users/Yuvalr/opt/anaconda3/lib/python3.7/site-packages/pyproj/crs/crs.py:68: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [29]:
np.round(100*biome_realm_area['Tropical and Subtropical Forests']/biome_realm_area['Tropical and Subtropical Forests'].sum())

WWF_REALM2
Afrotropic     15.0
Australasia     5.0
Indo-Malay     29.0
Nearctic        1.0
Neotropic      47.0
Oceania         0.0
Palearctic      2.0
Name: area, dtype: float64

In [25]:
# estimate the area of cropland and pasture in each biome/realm combination
crop_da = xr.open_rasterio('data/cropland.tif')
x = crop_da.rio.clip(aggregated_biomes_gpd.iloc[1].geometry,aggregated_biomes_gpd.crs)

NameError: name 'xr' is not defined

In [ ]:
def reproject_raster(raster,dst_crs,band=1):
    transform, width, height = calculate_default_transform(raster.crs, dst_crs, raster.width, raster.height, *raster.bounds)
    kwargs = raster.meta.copy()

    kwargs.update({
        'crs': dst_crs,
        'transform': transform,
        'width': width,
        'height': height
    })
    # out_np = raster.read(band).T
    out_np = np.zeros((height,width))
    reproject(
    source=rio.band(raster, band),
    destination=out_np,
    src_transform=raster.transform,
    src_crs=raster.crs,
    dst_transform=transform,
    dst_crs=dst_crs,
    resampling=Resampling.nearest)
    return out_np,kwargs

def intersect_shp_rs(x,raster,raster_params):
    return gpd.GeoDataFrame.from_features(rs.zonal_stats(x.geometry,raster,nodata=raster_params['nodata'],affine=raster_params['transform'],geojson_out=True,copy_properties=True,stats='mean'))['mean']

In [ ]:
crop_rs = rio.open('data/cropland.tif')
pasture_rs = rio.open('data/pasture.tif')
moll = '+proj=moll +lon_0=0 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs'

crop_rs_mol,crop_params = reproject_raster(crop_rs,moll)
pasture_rs_mol,pasture_params = reproject_raster(pasture_rs,moll)

In [ ]:
# mean_crop_gdf = gpd.GeoDataFrame.from_features(rs.zonal_stats(ecoregion_merged_gdf,crop_mol,nodata=crop_rs.profile['nodata'],affine=crop_params['transform'],geojson_out=True,copy_properties=True,stats='mean'))
# mean_pasture_gdf = gpd.GeoDataFrame.from_features(rs.zonal_stats(ecoregion_merged_gdf,pasture_mol,nodata=pasture_rs.profile['nodata'],affine=pasture_params['transform'],geojson_out=True,copy_properties=True,stats='mean'))

In [ ]:
mean_crop_biome_realm = aggregated_biomes_gpd_mol.apply(intersect_shp_rs,args=(crop_rs_mol,crop_params),axis=1)
mean_pasture_biome_realm = aggregated_biomes_gpd_mol.apply(intersect_shp_rs,args=(pasture_rs_mol,pasture_params),axis=1)
total_crop_biome_realm = mean_crop_biome_realm[0]*aggregated_biomes_gpd_mol.area
total_pasture_biome_realm = mean_pasture_biome_realm[0]*aggregated_biomes_gpd_mol.area

In [ ]:
crop_realms = pd.concat({'Crops': total_crop_biome_realm.unstack().sum()},names=['aggregated biome 1'])
pasture_realms = pd.concat({'Pasture': total_pasture_biome_realm.unstack().sum()},names=['aggregated biome 1'])

In [ ]:
biome_no_crop_pasture = pd.concat([biome_realm_area,total_crop_biome_realm,total_pasture_biome_realm],axis=1).fillna(0)
biome_no_crop_pasture = (biome_no_crop_pasture['area']-biome_no_crop_pasture[0]-biome_no_crop_pasture[1])
biome_realm_area_full = pd.concat([biome_no_crop_pasture,crop_realms,pasture_realms])

In [ ]:
biome_realm_area_full.name = 'area'

#### Use only the average values

In [ ]:
# determine on which biomes and taxa to perform the analysis - use only biomes and taxa for which we have data in more than 1 realm
site_biome_realm_counts = soil_site_data_mass_all_realm.groupby(['aggregated taxon','aggregated biome','WWF_REALM2']).site.nunique()
taxa_biome_for_analysis = site_biome_realm_counts.groupby(['aggregated taxon','aggregated biome']).count()
taxa_biome_for_analysis = taxa_biome_for_analysis[taxa_biome_for_analysis>1]

In [ ]:
# calculate realm specific means
biome_realm_mean = soil_site_data_mass_all_realm.groupby(['aggregated taxon','aggregated biome','WWF_REALM2'])['mass_B'].mean().reset_index('WWF_REALM2')
biome_mean = soil_site_data_mass_all_realm.groupby(['aggregated taxon','aggregated biome'])['mass_B'].mean()

# filter the data to contain only the relevant taxa/biomes
filt_biome_realm_mean = biome_realm_mean.loc[taxa_biome_for_analysis.index].reset_index()
filt_biome_mean = biome_mean.loc[taxa_biome_for_analysis.index]

# merge the area of each realm to calculate the total mass in each realm
filt_biome_realm_mean = filt_biome_realm_mean.merge(biome_realm_area_full.reset_index(),left_on=['aggregated biome','WWF_REALM2'],right_on=['aggregated biome 1','WWF_REALM2'],how='inner')
filt_biome_realm_mean['total_mass'] = filt_biome_realm_mean['mass_B']*filt_biome_realm_mean['area']
no_realm_total = (filt_biome_mean*filt_biome_realm_mean.groupby(['aggregated taxon','aggregated biome'])['area'].sum()).fillna(0).sum()

In [ ]:
print(no_realm_total/1e15)
print(filt_biome_realm_mean['total_mass'].sum()/1e15)

#### Perform the same analysis but using bootstrapping and Monte Carlo

In [ ]:
  
#Bootstrap using random choose with replacement. returns an array of means
def Boot_means(vals, sigma,Boot_N):
    L = len(vals)
    means = np.zeros(Boot_N) 

    for ii in range(Boot_N): #random choose with replacement
        indx = rng.choice(np.array(range(L)),L, replace=True)

        Boot = np.zeros(L)
        jj=0
        for ind in indx: #add the measurement uncertainties using a normal distribution
            Boot[jj] = np.max( [ 0 , rng.normal(vals[ind], sigma[ind], 1) ])
            jj +=1
        
        means[ii] = np.mean(Boot) #take the mean of a single resample
    
    return means

#Extract mean and 95% CI from bootstrapped data (returns an array)
def Boot_stats(means, prcnt_low=2.5, prcnt_high=97.5):    
    Mean = np.mean(means)
    prcnt = np.percentile(means,[prcnt_low,prcnt_high])
    return np.array([Mean , prcnt[0], prcnt[1]])


#Functions that help to handel the bootstrapped data and help in the Monte Carlo process for the total sum

#~~~ Think how to improve this function to reduce the effects of bin position and number of bins. Perhaps using a kde?

def means_to_cdf(means, Nbins=20): #converts the means array into a descrete cdf (cumulative distribution function) using a histogram with Nbins bins. 
    hist, bin_edges = np.histogram(means, bins = Nbins, density = False)
    bin_centers = (bin_edges[:-1] + bin_edges[1:] )/2
    cdf = np.cumsum(hist)
    cdf = cdf / cdf[-1]
    return cdf, bin_centers

def random_from_cdf(cdf,bin_centers,N): #picks N values of bin_centers according to the histogram's cdf
    values = np.random.rand(N)
    value_bins_indx = np.searchsorted(cdf, values)
    rnd_from_cdf = bin_centers[value_bins_indx]
    return rnd_from_cdf

rng = np.random.default_rng()

Calculate means by biome

In [ ]:
#extract the data for bootstrapping, and make the calculation using a for loop
x = soil_site_data_mass_all_realm.set_index(['aggregated taxon','aggregated biome'])['mass_G'].sort_index()
sigma = soil_site_data_mass_all_std_realm.set_index(['aggregated taxon','aggregated biome'])['mass_G'].sort_index()

soil_biome_mean_std_boot = pd.DataFrame(index = x.index.unique(), columns=['mean','2.5%','97.5%']) #initialize the bootstrap array for soil data

soil_means_dist = pd.DataFrame(index = x.index.unique(), columns=['cdf','values']) #initialize the means distribution array 

#Parameters for bootstrapping: (change to control precision and runtime)
NBins = 20
Boot_N = 1000

for taxon, biome in x.index.unique():
    means = Boot_means(x.loc[taxon,biome], sigma.loc[taxon,biome],Boot_N = Boot_N)
    soil_biome_mean_std_boot.loc[taxon,biome] = Boot_stats(means, prcnt_low=2.5, prcnt_high=97.5)
    soil_means_dist.loc[taxon,biome] = means_to_cdf(means, Nbins=NBins)

Filter data to contain only biomes relevant for the 

In [ ]:
#plot distribution of the total biomass using bootstrapping

biome_area = pd.read_csv('data/aggregated biomes data.csv') # biomes area in units of m^2
biome_area.loc[biome_area.biome=='Mediterranean Forests, Woodlands and Scrub','aggregated biome 1'] = 'Mediterranean Forests, Woodlands and Scrub'
biome_area1 = biome_area.groupby('aggregated biome 1')['area'].sum() #aggregate the biomes areas

filt_soil_means_dist = soil_means_dist.loc[taxa_biome_for_analysis.index]
# filt_soil_means_dist = soil_means_dist[~soil_means_dist.index.isin(taxa_biome_for_analysis.index)]
filt_soil_means_dist_tot = filt_soil_means_dist
filt_soil_means_dist_tot['total_mass'] = filt_soil_means_dist_tot['values']*filt_biome_realm_mean.groupby(['aggregated taxon','aggregated biome'])['area'].sum()
# filt_soil_means_dist_tot = filt_soil_means_dist.merge(pd.DataFrame(biome_area1),left_on='aggregated biome',right_index=True) #add the total areas of the aggregated biomes 
# soil_means_dist_tot = filt_soil_means_dist.merge(pd.DataFrame(biome_area1),left_on='aggregated biome',right_index=True) #add the total areas of the aggregated biomes 
# soil_means_dist_tot.loc[:,'values'] = (soil_means_dist_tot['values']*soil_means_dist_tot['area'])
# filt_soil_means_dist_tot['total_mass'] = filt_soil_means_dist_tot['values']*filt_soil_means_dist_tot['area']
# soil_means_dist_tot.drop('area',axis='columns',inplace=True)

N_total_dist = 10000
total_soil_dist = filt_soil_means_dist_tot.apply(lambda x: pd.Series(random_from_cdf(x['cdf'],x['total_mass'],N_total_dist),index=range(N_total_dist)),axis=1)
total_soil_dist /= 1e15

plt.hist(total_soil_dist.sum(axis=0),40);

In [ ]:
#extract the data for bootstrapping, and make the calculation using a for loop
x = soil_site_data_mass_all_realm.set_index(['aggregated taxon','aggregated biome','WWF_REALM2'])['mass_G'].sort_index()
sigma = soil_site_data_mass_all_std_realm.set_index(['aggregated taxon','aggregated biome','WWF_REALM2'])['mass_G'].sort_index()

soil_biome_realm_mean_std_boot = pd.DataFrame(index = x.index.unique(), columns=['mean','2.5%','97.5%']) #initialize the bootstrap array for soil data

soil_biome_realm_means_dist = pd.DataFrame(index = x.index.unique(), columns=['cdf','values']) #initialize the means distribution array 

#Parameters for bootstrapping: (change to control precision and runtime)
NBins = 20
Boot_N = 1000

for taxon, biome, realm in x.index.unique():
    means = Boot_means(x.loc[taxon,biome,realm], sigma.loc[taxon,biome,realm],Boot_N = Boot_N)
    soil_biome_realm_mean_std_boot.loc[taxon,biome,realm] = Boot_stats(means, prcnt_low=2.5, prcnt_high=97.5)
    soil_biome_realm_means_dist.loc[taxon,biome,realm] = means_to_cdf(means, Nbins=NBins)

In [ ]:
filt_soil_biome_realm_means_dist = soil_biome_realm_means_dist.reset_index('WWF_REALM2').loc[taxa_biome_for_analysis.index].reset_index()
filt_soil_biome_realm_means_dist = filt_soil_biome_realm_means_dist.merge(biome_realm_area_full.reset_index(),left_on=['aggregated biome','WWF_REALM2'],right_on=['aggregated biome 1','WWF_REALM2'],how='inner')
filt_soil_biome_realm_means_dist['total_mass'] = filt_soil_biome_realm_means_dist['values']*filt_soil_biome_realm_means_dist['area']
N_total_dist = 10000
total_soil_dist_realm = filt_soil_biome_realm_means_dist.apply(lambda x: pd.Series(random_from_cdf(x['cdf'],x['total_mass'],N_total_dist),index=range(N_total_dist)),axis=1)
total_soil_dist_realm /= 1e15


In [ ]:
fig, ax = plt.subplots(dpi=300,figsize=(3,3))
ax.hist(total_soil_dist.sum(axis=0),40,alpha=0.6,label='aggregated\nby biome',density=True);
ax.hist(total_soil_dist_realm.sum(),40,alpha=0.6,label='aggregated\nby realm',density=True);
ax.set(xlabel='Mt dry weight')
ax.legend()
ax.yaxis.set_major_formatter(ticker.PercentFormatter(1,1))
ax.set(ylabel='probability density')
plt.savefig('results/fig_s2.png',dpi=300,bbox_inches='tight')

In [ ]:
print(total_soil_dist.sum(axis=0).median())
print(total_soil_dist_realm.sum().median())

### Aggregating soil groups without biomes

In [ ]:
#extract the data for bootstrapping, and make the calculation using a for loop
x = soil_site_data_mass_all_realm.set_index(['aggregated taxon'])['mass_G'].sort_index()
sigma = soil_site_data_mass_all_std_realm.set_index(['aggregated taxon'])['mass_G'].sort_index()

soil_mean_std_boot_no_biome = pd.DataFrame(index = x.index.unique(), columns=['mean','2.5%','97.5%']) #initialize the bootstrap array for soil data

soil_means_dist_no_biome = pd.DataFrame(index = x.index.unique(), columns=['cdf','values']) #initialize the means distribution array 

#Parameters for bootstrapping: (change to control precision and runtime)
NBins = 20
Boot_N = 1000

for taxon in x.index.unique():
    means = Boot_means(x.loc[taxon], sigma.loc[taxon],Boot_N = Boot_N)
    soil_mean_std_boot_no_biome.loc[taxon] = Boot_stats(means, prcnt_low=2.5, prcnt_high=97.5)
    soil_means_dist_no_biome.loc[taxon] = means_to_cdf(means, Nbins=NBins)

In [ ]:
soil_means_dist_no_biome['total_mass'] = soil_means_dist_no_biome['values']*biome_realm_area.drop('Excluded').sum()
N_total_dist = 10000
total_soil_dist_no_biome = soil_means_dist_no_biome.apply(lambda x: pd.Series(random_from_cdf(x['cdf'],x['total_mass'],N_total_dist),index=range(N_total_dist)),axis=1)
total_soil_dist_no_biome /= 1e15


In [ ]:
soil_means_dist
soil_means_dist['total_mass'] = soil_means_dist['values'].values*biome_area1.loc[soil_means_dist.index.get_level_values(1)].values
N_total_dist = 10000
total_soil_dist = soil_means_dist.apply(lambda x: pd.Series(random_from_cdf(x['cdf'],x['total_mass'],N_total_dist),index=range(N_total_dist)),axis=1)
total_soil_dist /= 1e15


In [ ]:
fig, ax = plt.subplots(dpi=300,figsize=(3,3))
ax.hist(total_soil_dist.sum(axis=0),40,alpha=0.6,label='aggregated\nby biome',density=True);
ax.hist(total_soil_dist_no_biome.sum(),40,alpha=0.6,label='not aggregated',density=True);
ax.set(xlabel='Mt dry weight')
ax.legend()
ax.yaxis.set_major_formatter(ticker.PercentFormatter(1,1))
ax.set(ylabel='probability density')
plt.savefig('results/fig_s3.png',dpi=300,bbox_inches='tight')

In [ ]:
total_soil_dist.sum(axis=0).mean()

In [ ]:
import geemap
# Map = geemap.Map(center=(40, -100), zoom=4)
# Map

In [ ]:
Map = geemap.Map(center=(40, -100), zoom=4)